# <center> Model Gini Calculation </center>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

from sklearn.metrics import roc_auc_score
from datetime import datetime, timedelta
# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

# Function

## calculate_gini_for_threedigitscore

In [2]:
# def calculate_gini_for_threedigitscore(scores, labels):
#     """
#     Calculate Gini coefficient for three-digit scores and binary labels
    
#     Parameters:
#     scores: array-like, three-digit scores (higher is better)
#     labels: array-like, binary values (0 or 1, where 1 indicates default)
    
#     Returns:
#     float: Gini coefficient
#     """
#     # Combine scores and labels into a DataFrame
#     df = pd.DataFrame({'score': scores, 'label': labels})
    
#     # Sort by score in descending order (assuming higher score is better)
#     df = df.sort_values('score', ascending=False)
    
#     # Calculate cumulative values
#     total_pos = df['label'].sum()
#     total_neg = len(df) - total_pos
    
#     if total_pos == 0 or total_neg == 0:
#         return 0
    
#     # Calculate cumulative proportions
#     cum_pos = df['label'].cumsum()
#     cum_neg = np.arange(1, len(df) + 1) - cum_pos
    
#     # Convert to proportions
#     cum_pos_prop = cum_pos / total_pos
#     cum_neg_prop = cum_neg / total_neg
    
#     # Calculate Gini
#     gini = 1 - np.trapz(cum_pos_prop, cum_neg_prop)
    
#     return gini


## Modified one

def calculate_gini_for_threedigitscore(scores, labels):
    """
    Calculate Gini coefficient for three-digit scores and binary labels
    
    Parameters:
    scores: array-like, three-digit scores (higher is better)
    labels: array-like, binary values (0 or 1, where 1 indicates default)
    
    Returns:
    float: Gini coefficient
    """
    # Combine scores and labels into a DataFrame
    df = pd.DataFrame({'score': scores, 'label': labels})
    
    # Sort by score in descending order (assuming higher score means lower risk)
    # For default prediction, we want to sort scores in ascending order 
    # since higher default probability should correspond to higher risk
    df = df.sort_values('score', ascending=True)  # Changed to ascending=True
    
    # Calculate cumulative values
    total_pos = df['label'].sum()
    total_neg = len(df) - total_pos
    
    if total_pos == 0 or total_neg == 0:
        return 0
    
    # Calculate cumulative proportions
    cum_pos = df['label'].cumsum()
    cum_neg = np.arange(1, len(df) + 1) - cum_pos
    
    # Convert to proportions
    cum_pos_prop = cum_pos / total_pos
    cum_neg_prop = cum_neg / total_neg
    
    # Calculate area under curve
    auc = np.trapz(cum_pos_prop, cum_neg_prop)
    
    # Calculate Gini
    gini = 2 * auc - 1
    
    return gini

## calculate_gini

In [3]:
def calculate_gini(pd_scores, bad_indicators):
    """
    Calculate Gini coefficient from scores and binary indicators
    
    Parameters:
    pd_scores: array-like of scores/probabilities
    bad_indicators: array-like of binary outcomes (0/1)
    
    Returns:
    float: Gini coefficient
    """
    # Convert inputs to numpy arrays and ensure they're numeric
    pd_scores = np.array(pd_scores, dtype=float)
    bad_indicators = np.array(bad_indicators, dtype=int)
    
    # Check for valid input data
    if len(pd_scores) == 0 or len(bad_indicators) == 0:
        return np.nan
    
    # Check if we have both good and bad cases (needed for ROC AUC)
    if len(np.unique(bad_indicators)) < 2:
        return np.nan
    
    # Calculate AUC using sklearn
    try:
        auc = roc_auc_score(bad_indicators, pd_scores)
        # Calculate Gini from AUC
        gini = 2 * auc - 1
        return gini
    except ValueError:
        return np.nan

## calculate_hybrid_gini

In [4]:
# def calculate_hybrid_gini(scores, labels):
#     """
#     Calculate Gini coefficient handling both PD values and three-digit scores
    
#     Parameters:
#     scores: array-like, contains either PD values (0-1) or three-digit scores
#     labels: array-like, binary values (0 or 1, where 1 indicates default)
    
#     Returns:
#     float: Gini coefficient
#     """
#     # Convert inputs to numpy arrays
#     scores = np.array(scores, dtype=float)
#     labels = np.array(labels, dtype=int)
    
#     # Basic validation
#     if len(scores) == 0 or len(labels) == 0:
#         return np.nan
    
#     if len(np.unique(labels)) < 2:
#         return np.nan
        
#     # Determine if scores are PD values or three-digit scores
#     # PD values are between 0 and 1
#     is_pd = np.all((scores >= 0) & (scores <= 1))
    
#     if is_pd:
#         try:
#             auc = roc_auc_score(labels, scores)
#             gini = 2 * auc - 1
#             return gini
#         except ValueError:
#             return np.nan
#     else:
#         # Handle as three-digit score
#         df = pd.DataFrame({'score': scores, 'label': labels})
#         df = df.sort_values('score', ascending=False)
        
#         total_pos = df['label'].sum()
#         total_neg = len(df) - total_pos
        
#         if total_pos == 0 or total_neg == 0:
#             return np.nan
        
#         cum_pos = df['label'].cumsum()
#         cum_neg = np.arange(1, len(df) + 1) - cum_pos
        
#         cum_pos_prop = cum_pos / total_pos
#         cum_neg_prop = cum_neg / total_neg
        
#         gini = 1 - np.trapz(cum_pos_prop, cum_neg_prop)
#         return gini

## Modified one

def calculate_hybrid_gini(scores, labels):
    """
    Calculate Gini coefficient handling both PD values and three-digit scores
    
    Parameters:
    scores: array-like, contains either PD values (0-1) or three-digit scores
    labels: array-like, binary values (0 or 1, where 1 indicates default)
    
    Returns:
    float: Gini coefficient
    """
    # Convert inputs to numpy arrays
    scores = np.array(scores, dtype=float)
    labels = np.array(labels, dtype=int)
    
    # Basic validation
    if len(scores) == 0 or len(labels) == 0:
        return np.nan
    
    if len(np.unique(labels)) < 2:
        return np.nan
        
    # Determine if scores are PD values or three-digit scores
    # PD values are between 0 and 1
    is_pd = np.all((scores >= 0) & (scores <= 1))
    
    if is_pd:
        try:
            auc = roc_auc_score(labels, scores)
            gini = 2 * auc - 1
            return gini
        except ValueError:
            return np.nan
    else:
        # Handle as three-digit score
        df = pd.DataFrame({'score': scores, 'label': labels})
        # Sort by score in ascending order since higher score means higher risk
        df = df.sort_values('score', ascending=True)
        
        total_pos = df['label'].sum()
        total_neg = len(df) - total_pos
        
        if total_pos == 0 or total_neg == 0:
            return np.nan
        
        cum_pos = df['label'].cumsum()
        cum_neg = np.arange(1, len(df) + 1) - cum_pos
        
        cum_pos_prop = cum_pos / total_pos
        cum_neg_prop = cum_neg / total_neg
        
        # Calculate area under curve
        auc = np.trapz(cum_pos_prop, cum_neg_prop)
        
        # Calculate Gini using the same formula as PD values
        gini = 2 * auc - 1
        return gini

## calculate_periodic_gini_threedigit

In [5]:
# Main processing code
def calculate_periodic_gini_threedigit(df, score_column, label_column, namecolumn):
    """
    Calculate periodic Gini coefficients
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    """
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Calculate weekly Gini
    df['week'] = df['disbursementdate'].dt.to_period('W')
    weekly_gini = df.groupby('week').apply(
        lambda x: calculate_gini_for_threedigitscore(x[score_column], x[label_column])
    ).reset_index(name='gini')
    weekly_gini['period'] = 'Week'
    weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
    weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
    weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Calculate monthly Gini
    df['month'] = df['disbursementdate'].dt.to_period('M')
    monthly_gini = df.groupby('month').apply(
        lambda x: calculate_gini_for_threedigitscore(x[score_column], x[label_column])
    ).reset_index(name='gini')
    monthly_gini['period'] = 'Month'
    monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
    monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
    monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Combine and sort results
    gini_results = pd.concat([weekly_gini, monthly_gini])
    gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
    
    # Add metadata columns
    gini_results['Model_Name'] = score_column
    gini_results['version'] = '1.1.0'
    gini_results['bad_rate'] = namecolumn
    gini_results.rename(columns={'gini': f'{score_column}_{namecolumn}_gini'}, inplace=True)
    
    return gini_results

## calculate_periodic_gini

In [6]:
def calculate_periodic_gini(df, score_column, label_column, namecolumn):
    """
    Calculate periodic Gini coefficients
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    """
    # Input validation
    required_columns = ['disbursementdate', score_column, label_column]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
        
    # Create a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Ensure score and label columns are numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
    # Drop rows with invalid values
    df = df.dropna(subset=[score_column, label_column])
    
    # Calculate weekly Gini
    df['week'] = df['disbursementdate'].dt.to_period('W')
    weekly_gini = df.groupby('week').apply(
        lambda x: calculate_gini(x[score_column], x[label_column])
        if len(x) >= 10 else np.nan  # Only calculate if we have enough samples
    ).reset_index(name='gini')
    weekly_gini['period'] = 'Week'
    weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
    weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
    weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Calculate monthly Gini
    df['month'] = df['disbursementdate'].dt.to_period('M')
    monthly_gini = df.groupby('month').apply(
        lambda x: calculate_gini(x[score_column], x[label_column])
        if len(x) >= 20 else np.nan  # Only calculate if we have enough samples
    ).reset_index(name='gini')
    monthly_gini['period'] = 'Month'
    monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
    monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
    monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Combine and sort results
    gini_results = pd.concat([weekly_gini, monthly_gini])
    gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
    
    # Add metadata columns
    gini_results['Model_Name'] = score_column
    gini_results['version'] = '1.1.0'
    gini_results['bad_rate'] = namecolumn
    gini_results.rename(columns={'gini': f'{score_column}_{namecolumn}_gini'}, inplace=True)
    
    return gini_results

## calculate_periodic_hybrid_gini

In [7]:
def calculate_periodic_hybrid_gini(df, score_column, label_column, namecolumn):
    """
    Calculate periodic Gini coefficients for mixed score types
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    """
    # Input validation
    required_columns = ['disbursementdate', score_column, label_column]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
        
    # Create a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Ensure score and label columns are numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
    # Drop rows with invalid values
    df = df.dropna(subset=[score_column, label_column])
    
    # Calculate weekly Gini
    df['week'] = df['disbursementdate'].dt.to_period('W')
    weekly_gini = df.groupby('week').apply(
        lambda x: calculate_hybrid_gini(x[score_column], x[label_column])
        if len(x) >= 10 else np.nan  # Only calculate if we have enough samples
    ).reset_index(name='gini')
    weekly_gini['period'] = 'Week'
    weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
    weekly_gini['end_date'] = weekly_gini['start_date'] + pd.Timedelta(days=6)
    weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Calculate monthly Gini
    df['month'] = df['disbursementdate'].dt.to_period('M')
    monthly_gini = df.groupby('month').apply(
        lambda x: calculate_hybrid_gini(x[score_column], x[label_column])
        if len(x) >= 20 else np.nan  # Only calculate if we have enough samples
    ).reset_index(name='gini')
    monthly_gini['period'] = 'Month'
    monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
    monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
    monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Combine and sort results
    gini_results = pd.concat([weekly_gini, monthly_gini])
    gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
    
    # Add metadata columns
    gini_results['Model_Name'] = score_column
    gini_results['version'] = '1.1.0'
    gini_results['bad_rate'] = namecolumn
    gini_results.rename(columns={'gini': f'{score_column}_{namecolumn}_gini'}, inplace=True)
    
    return gini_results

# App Score FPD10

In [8]:
sq = """with appscore as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    apps_score ,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    apps_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from appscore;"""

dfappscorefpd10 = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 0880e235-f0cb-46d3-aafa-b6252508a69b successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [9]:
dfappscorefpd10.sample(5)

,disbursementdate,digitalLoanAccountId,apps_score,ln_fpd10_flag,ln_mature_fpd10_flag
17110,2024-12-31 12:56:22,1194f96c-ca70-4db5-8fd9-d3e904b5ba0f,0.570447,0,1
129845,2024-05-18 19:07:29,9b9c953b-1126-4d0f-b55f-f272cb391e05,0.350146,0,1
43124,2024-11-01 13:45:10,a8680bc7-5b8b-45de-bc2c-8b4d69366b84,0.621387,0,1
17931,2024-12-15 15:15:32,47021965-e17a-4d9b-a213-62da3fac000b,0.426387,1,1
87427,2024-04-15 16:25:14,24805191-9633-4c11-8ec6-36c88b7908a4,0.521484,0,1


In [11]:
gini_results = calculate_periodic_gini(dfappscorefpd10, 'apps_score', 'ln_fpd10_flag', 'FPD10')

In [12]:
gini_results.head()

,start_date,end_date,apps_score_FPD10_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.548230,Week,apps_score,1.1.0,FPD10
1,2023-06-01,2023-06-30,0.384648,Month,apps_score,1.1.0,FPD10
2,2023-06-05,2023-06-11,0.500000,Week,apps_score,1.1.0,FPD10
3,2023-06-12,2023-06-18,0.555195,Week,apps_score,1.1.0,FPD10
4,2023-06-19,2023-06-25,0.004831,Week,apps_score,1.1.0,FPD10


In [13]:
appscoreFPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{appscoreFPD10.shape}")
appscoreFPD10.columns.values

The shape of dataframe after copy is:	(108, 7)


array(['start_date', 'end_date', 'apps_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [14]:
gini_results.head()

,start_date,end_date,apps_score_FPD10_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.548230,Week,apps_score,1.1.0,FPD10
1,2023-06-01,2023-06-30,0.384648,Month,apps_score,1.1.0,FPD10
2,2023-06-05,2023-06-11,0.500000,Week,apps_score,1.1.0,FPD10
3,2023-06-12,2023-06-18,0.555195,Week,apps_score,1.1.0,FPD10
4,2023-06-19,2023-06-25,0.004831,Week,apps_score,1.1.0,FPD10


# App Score FPD30

In [15]:
sq = """
with appscore as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    apps_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    apps_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from appscore;
"""

dfappscorefpd30 = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID e0db7f34-f63e-4abb-9948-41ca2ad0d299 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [16]:
dfappscorefpd30.head()

,disbursementdate,digitalLoanAccountId,apps_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2024-09-01 15:18:37,1a2cb6f2-e66f-444a-ba22-a05cb5652ef2,0.514616,0,1
1,2024-12-15 12:58:01,950353da-8475-4d52-964b-e334dcbd95dc,0.658334,0,1
2,2024-06-19 20:08:28,210afa02-21e2-4ecc-8e28-06969222c968,0.596885,0,1
3,2024-07-22 18:45:38,9cfd101a-22e3-4f8f-9ce0-c1f1da881506,0.462215,0,1
4,2024-09-19 20:32:20,4f7bb0f5-ec5b-40c0-a145-6d8aa9b083ee,0.556686,1,1


In [18]:
gini_results = calculate_periodic_gini(dfappscorefpd30, 'apps_score', 'ln_fpd30_flag', 'FPD30')
# gini_results['bad_rate'] = 'FPD30'
appscoreFPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{appscoreFPD30.shape}")
appscoreFPD30.columns.values

The shape of dataframe after copy is:	(105, 7)


array(['start_date', 'end_date', 'apps_score_FPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [19]:
gini_results.sample(5)

,start_date,end_date,apps_score_FPD30_gini,period,Model_Name,version,bad_rate
2,2023-06-05,2023-06-11,0.451168,Week,apps_score,1.1.0,FPD30
3,2023-06-12,2023-06-18,0.521893,Week,apps_score,1.1.0,FPD30
40,2024-01-08,2024-01-14,0.434836,Week,apps_score,1.1.0,FPD30
83,2024-09-09,2024-09-15,0.285449,Week,apps_score,1.1.0,FPD30
18,2023-09-04,2023-09-10,0.290494,Week,apps_score,1.1.0,FPD30


# App Score FSPD30

In [20]:
sq = """with appscore as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    apps_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    apps_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from appscore;
"""

dfappscorefspd30 = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 35ae93ec-228a-40d3-ac3e-2fc6a54e18f2 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [21]:
gini_results = calculate_periodic_gini(dfappscorefspd30, 'apps_score', 'ln_fspd30_flag', 'FSPD30')
# gini_results['bad_rate'] = 'FSPD30'
appscoreFSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{appscoreFSPD30.shape}")
appscoreFSPD30.columns.values

The shape of dataframe after copy is:	(100, 7)


array(['start_date', 'end_date', 'apps_score_FSPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [22]:
gini_results.tail()

,start_date,end_date,apps_score_FSPD30_gini,period,Model_Name,version,bad_rate
95,2024-11-18,2024-11-24,0.302444,Week,apps_score,1.1.0,FSPD30
96,2024-11-25,2024-12-01,0.260883,Week,apps_score,1.1.0,FSPD30
97,2024-12-01,2024-12-31,0.311393,Month,apps_score,1.1.0,FSPD30
98,2024-12-02,2024-12-08,0.309329,Week,apps_score,1.1.0,FSPD30
99,2024-12-09,2024-12-15,0.261535,Week,apps_score,1.1.0,FSPD30


# App Score FSTPD30

In [23]:
sq = """with appscore as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    apps_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    apps_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from appscore;
"""

dfappscorefstpd30 = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID ba37f496-4893-4d3a-a7dd-9d4f2e5f7009 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [24]:
gini_results = calculate_periodic_gini(dfappscorefstpd30, 'apps_score', 'ln_fstpd30_flag', 'FSTPD30')
# gini_results['bad_rate'] = 'FSTPD30'
appscoreFSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{appscoreFSTPD30.shape}")
appscoreFSTPD30.columns.values

The shape of dataframe after copy is:	(94, 7)


array(['start_date', 'end_date', 'apps_score_FSTPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [25]:
appscoreFSTPD30.head()

,start_date,end_date,apps_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.465098,Week,apps_score,1.1.0,FSTPD30
1,2023-06-01,2023-06-30,0.337420,Month,apps_score,1.1.0,FSTPD30
2,2023-06-05,2023-06-11,0.413300,Week,apps_score,1.1.0,FSTPD30
3,2023-06-12,2023-06-18,0.395676,Week,apps_score,1.1.0,FSTPD30
4,2023-06-19,2023-06-25,0.314497,Week,apps_score,1.1.0,FSTPD30


# Combining App Score

In [26]:
import functools

dataframes = [appscoreFPD10, appscoreFPD30, appscoreFSPD30, appscoreFSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'apps_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'apps_score_FPD30_gini',
       'apps_score_FSPD30_gini', 'apps_score_FSTPD30_gini'], dtype=object)

In [30]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate','apps_score_FSTPD30_gini','apps_score_FSPD30_gini',
       'apps_score_FPD30_gini', 'apps_score_FPD10_gini']].copy()
final_df.dtypes

start_date                 datetime64[ns]
end_date                   datetime64[ns]
period                             object
Model_Name                         object
version                            object
bad_rate                           object
apps_score_FSTPD30_gini           float64
apps_score_FSPD30_gini            float64
apps_score_FPD30_gini             float64
apps_score_FPD10_gini             float64
dtype: object

## Creating app score table 

In [31]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_s_apps_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=d7e4a0cd-3d8a-4d90-8dac-2c1bf74205d4>

In [32]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('apps_score_FSTPD30_gini', 'FLOAT'),
    bigquery.SchemaField('apps_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('apps_score_FPD30_gini', 'FLOAT'),
    bigquery.SchemaField('apps_score_fpd10_gini', 'FLOAT')
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_s_apps_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=62d079f9-20af-4b73-97f1-15f8ff64415c>

# sb_demo_score

## FPD10

In [33]:
sq = """
with sb_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    beta_demo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    beta_demo_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from sb_demo_score;
"""

df_sb_demo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 1b849be7-cfef-45a2-ba19-c3ee7c6e0af6 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [34]:
df_sb_demo_scorefpd10.head()

,disbursementdate,digitalLoanAccountId,beta_demo_score,ln_fpd10_flag,ln_mature_fpd10_flag
0,2024-07-16 19:09:27,a731f941-ed8a-4278-9ba4-1fd41132aa5e,0.196001,0,1
1,2023-12-03 17:15:55,8effa90f-b747-40b9-b42f-39691224f596,0.114995,1,1
2,2024-02-26 15:08:47,acebf8de-bd42-4b00-83bc-d7c0999a7488,0.075880,0,1
3,2024-05-02 17:24:11,9b118965-74ea-4832-a640-8b5ea2cc140e,0.088430,0,1
4,2024-09-15 09:21:50,2f8156e6-5c5b-454c-92d9-628291e6f99e,0.075348,0,1


In [35]:
gini_results = calculate_periodic_gini(df_sb_demo_scorefpd10, 'beta_demo_score', 'ln_fpd10_flag', 'FPD10')
sb_demo_scoreFPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{sb_demo_scoreFPD10.shape}")
sb_demo_scoreFPD10.columns.values

The shape of dataframe after copy is:	(108, 7)


array(['start_date', 'end_date', 'beta_demo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [36]:
sb_demo_scoreFPD10.tail()

,start_date,end_date,beta_demo_score_FPD10_gini,period,Model_Name,version,bad_rate
103,2025-01-01,2025-01-31,0.213186,Month,beta_demo_score,1.1.0,FPD10
104,2025-01-06,2025-01-12,0.186708,Week,beta_demo_score,1.1.0,FPD10
105,2025-01-13,2025-01-19,0.211580,Week,beta_demo_score,1.1.0,FPD10
106,2025-01-20,2025-01-26,0.172097,Week,beta_demo_score,1.1.0,FPD10
107,2025-01-27,2025-02-02,0.351495,Week,beta_demo_score,1.1.0,FPD10


## FPD30

In [37]:
sq = """
with sb_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    beta_demo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    beta_demo_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from sb_demo_score;
"""

df_sb_demo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID bfebfdfa-0c1a-417f-af5b-4e787b9bdd35 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [38]:
gini_results = calculate_periodic_gini(df_sb_demo_scorefpd30, 'beta_demo_score', 'ln_fpd30_flag', 'FPD30')
sb_demo_scoreFPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{sb_demo_scoreFPD30.shape}")
sb_demo_scoreFPD30.columns.values

The shape of dataframe after copy is:	(105, 7)


array(['start_date', 'end_date', 'beta_demo_score_FPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [39]:
sb_demo_scoreFPD30.head() 

,start_date,end_date,beta_demo_score_FPD30_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.289091,Week,beta_demo_score,1.1.0,FPD30
1,2023-06-01,2023-06-30,0.018045,Month,beta_demo_score,1.1.0,FPD30
2,2023-06-05,2023-06-11,0.207407,Week,beta_demo_score,1.1.0,FPD30
3,2023-06-12,2023-06-18,-0.073007,Week,beta_demo_score,1.1.0,FPD30
4,2023-06-19,2023-06-25,-0.243641,Week,beta_demo_score,1.1.0,FPD30


## FSPD30

In [40]:
sq = """
with sb_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    beta_demo_score,
    ln_fspd30_flag,   -- fspd30
	ln_mature_fspd30_flag,	--- fspd30 observation
	FROM 
   risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    beta_demo_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from sb_demo_score;
"""

df_sb_demo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 0d283ae1-e8e1-4b56-87d2-bdca2fa1c3f9 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [41]:
gini_results = calculate_periodic_gini(df_sb_demo_scorefspd30, 'beta_demo_score', 'ln_fspd30_flag', 'FSPD30')
sb_demo_scoreFSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{sb_demo_scoreFSPD30.shape}")
sb_demo_scoreFSPD30.columns.values

The shape of dataframe after copy is:	(100, 7)


array(['start_date', 'end_date', 'beta_demo_score_FSPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [42]:
sb_demo_scoreFSPD30.head()

,start_date,end_date,beta_demo_score_FSPD30_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.087601,Week,beta_demo_score,1.1.0,FSPD30
1,2023-06-01,2023-06-30,0.034373,Month,beta_demo_score,1.1.0,FSPD30
2,2023-06-05,2023-06-11,0.239192,Week,beta_demo_score,1.1.0,FSPD30
3,2023-06-12,2023-06-18,-0.153216,Week,beta_demo_score,1.1.0,FSPD30
4,2023-06-19,2023-06-25,0.077598,Week,beta_demo_score,1.1.0,FSPD30


## FSTPD30

In [43]:
sq = """
with sb_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    beta_demo_score ,
    ln_fstpd30_flag,   -- fstpd30
	ln_mature_fstpd30_flag,	--- fstpd30 observation
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    beta_demo_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from sb_demo_score;
"""

df_sb_demo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 8783a3e0-2190-40ba-af1f-06d1025d39be successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [44]:
gini_results = calculate_periodic_gini(df_sb_demo_scorefstpd30, 'beta_demo_score', 'ln_fstpd30_flag', 'FSTPD30')
sb_demo_scoreFSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{sb_demo_scoreFSTPD30.shape}")
sb_demo_scoreFSTPD30.columns.values

The shape of dataframe after copy is:	(94, 7)


array(['start_date', 'end_date', 'beta_demo_score_FSTPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [45]:
sb_demo_scoreFSTPD30.head()

,start_date,end_date,beta_demo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.210240,Week,beta_demo_score,1.1.0,FSTPD30
1,2023-06-01,2023-06-30,0.007692,Month,beta_demo_score,1.1.0,FSTPD30
2,2023-06-05,2023-06-11,0.291754,Week,beta_demo_score,1.1.0,FSTPD30
3,2023-06-12,2023-06-18,-0.291503,Week,beta_demo_score,1.1.0,FSTPD30
4,2023-06-19,2023-06-25,0.098133,Week,beta_demo_score,1.1.0,FSTPD30


# Combining sb demo score

In [46]:
import functools

dataframes = [sb_demo_scoreFPD10, sb_demo_scoreFPD30, sb_demo_scoreFSPD30, sb_demo_scoreFSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'beta_demo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'beta_demo_score_FPD30_gini',
       'beta_demo_score_FSPD30_gini', 'beta_demo_score_FSTPD30_gini'],
      dtype=object)

In [47]:
final_df = final_df[['start_date', 'end_date', 'period',
       'Model_Name', 'version', 'bad_rate','beta_demo_score_FPD10_gini','beta_demo_score_FPD30_gini',
       'beta_demo_score_FSPD30_gini', 'beta_demo_score_FSTPD30_gini']].copy()

## creating sb demo score table 

In [48]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_sb_demo_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=38bd9bea-b7b7-4ecb-aa31-753d2ba54696>

In [49]:


import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('beta_demo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('beta_demo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('beta_demo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('beta_demo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_sb_demo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=5b324882-518b-4360-8e69-d8727aee93a1>

# s_cic_score

## FPD10

In [50]:
sq = """
with s_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    cic_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    cic_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from s_cic_score;
"""

df_s_cic_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID a93ca437-1aec-46e2-915f-6783f45f5212 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [51]:
df_s_cic_scorefpd10.head()

,disbursementdate,digitalLoanAccountId,cic_score,ln_fpd10_flag,ln_mature_fpd10_flag
0,2024-07-16 19:09:27,a731f941-ed8a-4278-9ba4-1fd41132aa5e,0.117005,0,1
1,2024-05-02 17:24:11,9b118965-74ea-4832-a640-8b5ea2cc140e,0.125541,0,1
2,2024-09-15 09:21:50,2f8156e6-5c5b-454c-92d9-628291e6f99e,0.122660,0,1
3,2024-12-29 18:02:03,6548e088-8d8e-47ae-b33d-e6df00d80c38,0.058266,0,1
4,2025-01-27 12:33:16,107d411b-cd24-4dde-86a8-6f7eeeaaa7bb,0.126979,0,1


In [52]:
gini_results = calculate_periodic_gini(df_s_cic_scorefpd10, 'cic_score', 'ln_fpd10_flag', 'FPD10')
s_cic_scoreFPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{s_cic_scoreFPD10.shape}")
s_cic_scoreFPD10.columns.values

The shape of dataframe after copy is:	(107, 7)


array(['start_date', 'end_date', 'cic_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [53]:
s_cic_scoreFPD10.tail()

,start_date,end_date,cic_score_FPD10_gini,period,Model_Name,version,bad_rate
102,2025-01-01,2025-01-31,0.183128,Month,cic_score,1.1.0,FPD10
103,2025-01-06,2025-01-12,0.190109,Week,cic_score,1.1.0,FPD10
104,2025-01-13,2025-01-19,0.141473,Week,cic_score,1.1.0,FPD10
105,2025-01-20,2025-01-26,0.164915,Week,cic_score,1.1.0,FPD10
106,2025-01-27,2025-02-02,0.209019,Week,cic_score,1.1.0,FPD10


## FPD30

In [54]:
sq = """
with s_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    cic_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
  risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    cic_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from s_cic_score;
"""

df_s_cic_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID b65545ac-ae3c-44dd-bb99-fed986397b52 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [55]:
df_s_cic_scorefpd30.head()

,disbursementdate,digitalLoanAccountId,cic_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2024-07-16 19:09:27,a731f941-ed8a-4278-9ba4-1fd41132aa5e,0.117005,0,1
1,2024-05-02 17:24:11,9b118965-74ea-4832-a640-8b5ea2cc140e,0.125541,0,1
2,2024-09-15 09:21:50,2f8156e6-5c5b-454c-92d9-628291e6f99e,0.122660,0,1
3,2024-12-29 18:02:03,6548e088-8d8e-47ae-b33d-e6df00d80c38,0.058266,0,1
4,2024-07-01 14:17:27,571c6877-62e0-4c47-a382-77b1aeb2840c,0.126872,0,1


In [56]:
gini_results = calculate_periodic_gini(df_s_cic_scorefpd30, 'cic_score', 'ln_fpd30_flag', 'FPD30')
s_cic_scoreFPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{s_cic_scoreFPD30.shape}")
s_cic_scoreFPD30.columns.values

The shape of dataframe after copy is:	(104, 7)


array(['start_date', 'end_date', 'cic_score_FPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [57]:
s_cic_scoreFPD30.head()

,start_date,end_date,cic_score_FPD30_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.261538,Week,cic_score,1.1.0,FPD30
1,2023-06-01,2023-06-30,0.082209,Month,cic_score,1.1.0,FPD30
2,2023-06-05,2023-06-11,0.196970,Week,cic_score,1.1.0,FPD30
3,2023-06-12,2023-06-18,-0.617371,Week,cic_score,1.1.0,FPD30
4,2023-06-19,2023-06-25,0.435484,Week,cic_score,1.1.0,FPD30


In [58]:
s_cic_scoreFPD30.describe()

,start_date,end_date,cic_score_FPD30_gini
count,104,104,104.000000
mean,2024-03-20 01:36:55.384615424,2024-03-30 14:18:27.692307712,0.254210
min,2023-05-29 00:00:00,2023-06-04 00:00:00,-0.617371
25%,2023-10-28 06:00:00,2023-11-03 18:00:00,0.192568
50%,2024-03-21 12:00:00,2024-03-31 00:00:00,0.265936
75%,2024-08-13 18:00:00,2024-08-26 12:00:00,0.324917
max,2025-01-06 00:00:00,2025-01-31 00:00:00,0.749150
std,NaN,NaN,0.142915


## FSPD30

In [59]:
sq = """
with s_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    cic_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    cic_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from s_cic_score;
"""

df_s_cic_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 07dba652-3166-4955-abfa-1e520241e8ea successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [60]:
gini_results = calculate_periodic_gini(df_s_cic_scorefspd30, 'cic_score', 'ln_fspd30_flag', 'FSPD30')
s_cic_scoreFSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{s_cic_scoreFSPD30.shape}")
s_cic_scoreFSPD30.columns.values

The shape of dataframe after copy is:	(99, 7)


array(['start_date', 'end_date', 'cic_score_FSPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [61]:
s_cic_scoreFSPD30.describe()

,start_date,end_date,cic_score_FSPD30_gini
count,99,99,99.000000
mean,2024-03-05 19:09:05.454545408,2024-03-16 07:30:54.545454592,0.266398
min,2023-05-29 00:00:00,2023-06-04 00:00:00,-0.014423
25%,2023-10-19 12:00:00,2023-10-30 00:00:00,0.219232
50%,2024-03-04 00:00:00,2024-03-17 00:00:00,0.275248
75%,2024-07-25 12:00:00,2024-08-02 00:00:00,0.313029
max,2024-12-09 00:00:00,2024-12-31 00:00:00,0.545299
std,NaN,NaN,0.085948


In [62]:
s_cic_scoreFSPD30.tail()

,start_date,end_date,cic_score_FSPD30_gini,period,Model_Name,version,bad_rate
94,2024-11-18,2024-11-24,0.247155,Week,cic_score,1.1.0,FSPD30
95,2024-11-25,2024-12-01,0.286385,Week,cic_score,1.1.0,FSPD30
96,2024-12-01,2024-12-31,0.297500,Month,cic_score,1.1.0,FSPD30
97,2024-12-02,2024-12-08,0.279570,Week,cic_score,1.1.0,FSPD30
98,2024-12-09,2024-12-15,0.402519,Week,cic_score,1.1.0,FSPD30


## FSTPD30

In [63]:
sq = """
with s_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    cic_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    cic_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from s_cic_score;
"""

df_s_cic_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 3953d199-3b04-4f0b-a71a-96567797e7d7 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [64]:
gini_results = calculate_periodic_gini(df_s_cic_scorefstpd30, 'cic_score', 'ln_fstpd30_flag', 'FSTPD30')
s_cic_scoreFSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{s_cic_scoreFSTPD30.shape}")
s_cic_scoreFSTPD30.columns.values

The shape of dataframe after copy is:	(93, 7)


array(['start_date', 'end_date', 'cic_score_FSTPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [65]:
s_cic_scoreFSTPD30.describe()

,start_date,end_date,cic_score_FSTPD30_gini
count,93,93,93.000000
mean,2024-02-17 16:30:58.064516096,2024-02-28 05:40:38.709677312,0.249316
min,2023-05-29 00:00:00,2023-06-04 00:00:00,-0.164509
25%,2023-10-09 00:00:00,2023-10-22 00:00:00,0.218096
50%,2024-02-19 00:00:00,2024-02-29 00:00:00,0.263496
75%,2024-07-01 00:00:00,2024-07-07 00:00:00,0.304424
max,2024-11-04 00:00:00,2024-11-30 00:00:00,0.479082
std,NaN,NaN,0.089857


In [66]:
s_cic_scoreFSTPD30.tail()

,start_date,end_date,cic_score_FSTPD30_gini,period,Model_Name,version,bad_rate
88,2024-10-14,2024-10-20,0.283938,Week,cic_score,1.1.0,FSTPD30
89,2024-10-21,2024-10-27,0.267940,Week,cic_score,1.1.0,FSTPD30
90,2024-10-28,2024-11-03,0.225835,Week,cic_score,1.1.0,FSTPD30
91,2024-11-01,2024-11-30,0.295004,Month,cic_score,1.1.0,FSTPD30
92,2024-11-04,2024-11-10,0.327550,Week,cic_score,1.1.0,FSTPD30


# Combining s_cic_score

In [67]:
import functools

dataframes = [s_cic_scoreFPD10, s_cic_scoreFPD30, s_cic_scoreFSPD30, s_cic_scoreFSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'cic_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'cic_score_FPD30_gini',
       'cic_score_FSPD30_gini', 'cic_score_FSTPD30_gini'], dtype=object)

In [68]:
final_df = final_df[['start_date', 'end_date', 'period',
       'Model_Name', 'version', 'bad_rate','cic_score_FPD10_gini','cic_score_FPD30_gini', 'cic_score_FSPD30_gini', 'cic_score_FSTPD30_gini']].copy()

## Creating the table

In [69]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_s_cic_score;"""

client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=875ca2db-2249-4701-907a-ae67683892ec>

In [70]:


import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('cic_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('cic_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('cic_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('cic_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_s_cic_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=4aa66d24-45ac-4e5b-9de5-481d781dabbd>

# sb_stack_score

## FPD10

In [71]:
# sb_stack_score

sq = """
with sb_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    beta_stack_score beta_stack_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    beta_stack_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from sb_stack_score;
"""

df_sb_stack_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

df_sb_stack_scorefpd10.head()

Job ID f281d591-5490-492c-a58f-725baa6b9c86 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,beta_stack_score,ln_fpd10_flag,ln_mature_fpd10_flag
0,2024-09-01 15:18:37,1a2cb6f2-e66f-444a-ba22-a05cb5652ef2,0.063497,0,1
1,2024-12-15 12:58:01,950353da-8475-4d52-964b-e334dcbd95dc,0.232534,0,1
2,2024-07-27 19:43:54,cae2c508-fe59-47b9-8eef-fc5f6f08a184,0.055323,0,1
3,2024-06-19 20:08:28,210afa02-21e2-4ecc-8e28-06969222c968,0.160364,0,1
4,2024-07-22 18:45:38,9cfd101a-22e3-4f8f-9ce0-c1f1da881506,0.039724,0,1


In [72]:
gini_results = calculate_periodic_gini(df_sb_stack_scorefpd10, 'beta_stack_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

The shape of dataframe after copy is:	(108, 7)


array(['start_date', 'end_date', 'beta_stack_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [73]:
M1FPD10.describe()

,start_date,end_date,beta_stack_score_FPD10_gini
count,108,108,108.000000
mean,2024-03-26 08:13:20,2024-04-05 16:53:20,0.341678
min,2023-05-29 00:00:00,2023-06-04 00:00:00,-0.030303
25%,2023-10-28 06:00:00,2023-11-03 18:00:00,0.294745
50%,2024-03-28 12:00:00,2024-04-03 12:00:00,0.324847
75%,2024-08-27 12:00:00,2024-09-02 18:00:00,0.412945
max,2025-01-27 00:00:00,2025-02-02 00:00:00,0.557778
std,NaN,NaN,0.093908


In [74]:
M1FPD10.head()

,start_date,end_date,beta_stack_score_FPD10_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.506255,Week,beta_stack_score,1.1.0,FPD10
1,2023-06-01,2023-06-30,0.253923,Month,beta_stack_score,1.1.0,FPD10
2,2023-06-05,2023-06-11,0.407799,Week,beta_stack_score,1.1.0,FPD10
3,2023-06-12,2023-06-18,0.538580,Week,beta_stack_score,1.1.0,FPD10
4,2023-06-19,2023-06-25,-0.030303,Week,beta_stack_score,1.1.0,FPD10


## FPD30

In [75]:
# sb_stack_score

sq = """
with sb_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    beta_stack_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    beta_stack_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from sb_stack_score;
"""

df_sb_stack_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

df_sb_stack_scorefpd30.head()

Job ID 2bcf9779-b8fd-439e-80fe-3cf04e3473ad successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,beta_stack_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2024-09-01 15:18:37,1a2cb6f2-e66f-444a-ba22-a05cb5652ef2,0.063497,0,1
1,2024-12-15 12:58:01,950353da-8475-4d52-964b-e334dcbd95dc,0.232534,0,1
2,2024-07-27 19:43:54,cae2c508-fe59-47b9-8eef-fc5f6f08a184,0.055323,0,1
3,2024-06-19 20:08:28,210afa02-21e2-4ecc-8e28-06969222c968,0.160364,0,1
4,2024-07-22 18:45:38,9cfd101a-22e3-4f8f-9ce0-c1f1da881506,0.039724,0,1


In [76]:
gini_results = calculate_periodic_gini(df_sb_stack_scorefpd30, 'beta_stack_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

The shape of dataframe after copy is:	(105, 7)


array(['start_date', 'end_date', 'beta_stack_score_FPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [77]:
M2FPD30.describe()

,start_date,end_date,beta_stack_score_FPD30_gini
count,105,105,105.000000
mean,2024-03-17 18:44:34.285714176,2024-03-28 06:24:00,0.360785
min,2023-05-29 00:00:00,2023-06-04 00:00:00,-0.036145
25%,2023-10-23 00:00:00,2023-10-31 00:00:00,0.314057
50%,2024-03-18 00:00:00,2024-03-31 00:00:00,0.351709
75%,2024-08-12 00:00:00,2024-08-25 00:00:00,0.423529
max,2025-01-06 00:00:00,2025-01-31 00:00:00,0.564768
std,NaN,NaN,0.094090


In [78]:
M2FPD30.tail()

,start_date,end_date,beta_stack_score_FPD30_gini,period,Model_Name,version,bad_rate
100,2024-12-16,2024-12-22,0.306499,Week,beta_stack_score,1.1.0,FPD30
101,2024-12-23,2024-12-29,0.331995,Week,beta_stack_score,1.1.0,FPD30
102,2024-12-30,2025-01-05,0.289278,Week,beta_stack_score,1.1.0,FPD30
103,2025-01-01,2025-01-31,0.276003,Month,beta_stack_score,1.1.0,FPD30
104,2025-01-06,2025-01-12,0.324793,Week,beta_stack_score,1.1.0,FPD30


## FSPD30

In [79]:
# sb_stack_score

sq = """
with sb_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    beta_stack_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    beta_stack_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from sb_stack_score;
"""

df_sb_stack_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

df_sb_stack_scorefspd30.head()

Job ID be2e1c07-9b5f-4aca-8f32-747eab92de9b successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,beta_stack_score,ln_fspd30_flag,ln_mature_fspd30_flag
0,2024-07-16 19:09:27,a731f941-ed8a-4278-9ba4-1fd41132aa5e,0.107962,0,1
1,2024-02-26 15:08:47,acebf8de-bd42-4b00-83bc-d7c0999a7488,0.060378,0,1
2,2024-05-02 17:24:11,9b118965-74ea-4832-a640-8b5ea2cc140e,0.011423,0,1
3,2024-09-15 09:21:50,2f8156e6-5c5b-454c-92d9-628291e6f99e,0.075346,0,1
4,2024-10-13 17:33:13,c58cd395-1d70-4ec0-91c5-8a067645c73c,0.044324,0,1


In [80]:
gini_results = calculate_periodic_gini(df_sb_stack_scorefspd30, 'beta_stack_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
M3FSPD30.columns.values

The shape of dataframe after copy is:	(100, 7)


array(['start_date', 'end_date', 'beta_stack_score_FSPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [81]:
M3FSPD30.describe()

,start_date,end_date,beta_stack_score_FSPD30_gini
count,100,100,100.000000
mean,2024-03-03 12:57:36,2024-03-14 00:14:24,0.363951
min,2023-05-29 00:00:00,2023-06-04 00:00:00,0.179581
25%,2023-10-14 06:00:00,2023-10-27 06:00:00,0.315375
50%,2024-03-02 12:00:00,2024-03-13 12:00:00,0.346307
75%,2024-07-23 18:00:00,2024-08-01 00:00:00,0.423752
max,2024-12-09 00:00:00,2024-12-31 00:00:00,0.551886
std,NaN,NaN,0.077645


In [82]:
M3FSPD30.tail()

,start_date,end_date,beta_stack_score_FSPD30_gini,period,Model_Name,version,bad_rate
95,2024-11-18,2024-11-24,0.314577,Week,beta_stack_score,1.1.0,FSPD30
96,2024-11-25,2024-12-01,0.300199,Week,beta_stack_score,1.1.0,FSPD30
97,2024-12-01,2024-12-31,0.321988,Month,beta_stack_score,1.1.0,FSPD30
98,2024-12-02,2024-12-08,0.317909,Week,beta_stack_score,1.1.0,FSPD30
99,2024-12-09,2024-12-15,0.327067,Week,beta_stack_score,1.1.0,FSPD30


## FSTPD30

In [83]:
# sb_stack_score

sq = """
with sb_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    beta_stack_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    beta_stack_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from sb_stack_score;
"""

df_sb_stack_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

df_sb_stack_scorefstpd30.head()

Job ID c02d4f7b-da2b-419f-b513-a13fc15f6ec0 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,beta_stack_score,ln_fstpd30_flag,ln_mature_fstpd30_flag
0,2024-09-01 15:18:37,1a2cb6f2-e66f-444a-ba22-a05cb5652ef2,0.063497,0,1
1,2024-07-27 19:43:54,cae2c508-fe59-47b9-8eef-fc5f6f08a184,0.055323,0,1
2,2024-06-19 20:08:28,210afa02-21e2-4ecc-8e28-06969222c968,0.160364,0,1
3,2024-07-22 18:45:38,9cfd101a-22e3-4f8f-9ce0-c1f1da881506,0.039724,0,1
4,2024-07-26 11:14:26,e42d366d-9370-4761-9d94-e7cee5834849,0.092918,0,1


In [84]:
gini_results = calculate_periodic_gini(df_sb_stack_scorefstpd30, 'beta_stack_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
M4FSTPD30.columns.values

The shape of dataframe after copy is:	(94, 7)


array(['start_date', 'end_date', 'beta_stack_score_FSTPD30_gini',
       'period', 'Model_Name', 'version', 'bad_rate'], dtype=object)

In [85]:
M4FSTPD30.describe()

,start_date,end_date,beta_stack_score_FSTPD30_gini
count,94,94,94.000000
mean,2024-02-15 11:14:02.553191424,2024-02-25 23:14:02.553191424,0.338183
min,2023-05-29 00:00:00,2023-06-04 00:00:00,0.160057
25%,2023-10-03 18:00:00,2023-10-16 18:00:00,0.297767
50%,2024-02-15 12:00:00,2024-02-27 00:00:00,0.334322
75%,2024-06-29 06:00:00,2024-07-05 06:00:00,0.385845
max,2024-11-04 00:00:00,2024-11-30 00:00:00,0.500133
std,NaN,NaN,0.066316


In [86]:
M4FSTPD30.tail()

,start_date,end_date,beta_stack_score_FSTPD30_gini,period,Model_Name,version,bad_rate
89,2024-10-14,2024-10-20,0.275123,Week,beta_stack_score,1.1.0,FSTPD30
90,2024-10-21,2024-10-27,0.298238,Week,beta_stack_score,1.1.0,FSTPD30
91,2024-10-28,2024-11-03,0.287414,Week,beta_stack_score,1.1.0,FSTPD30
92,2024-11-01,2024-11-30,0.326381,Month,beta_stack_score,1.1.0,FSTPD30
93,2024-11-04,2024-11-10,0.305393,Week,beta_stack_score,1.1.0,FSTPD30


## Combining the dataframes

In [87]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'beta_stack_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'beta_stack_score_FPD30_gini',
       'beta_stack_score_FSPD30_gini', 'beta_stack_score_FSTPD30_gini'],
      dtype=object)

In [88]:
final_df = final_df[['start_date', 'end_date', 'period', 'Model_Name', 'version', 'bad_rate', 'beta_stack_score_FPD10_gini', 'beta_stack_score_FPD30_gini',
                     'beta_stack_score_FSPD30_gini', 'beta_stack_score_FSTPD30_gini']].copy()

## Creating the table 

In [89]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_sb_stack_score;"""

client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=65f9d7f7-c5e5-4b3e-b93b-e82e0235be68>

In [90]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('beta_stack_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('beta_stack_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('beta_stack_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('beta_stack_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_sb_stack_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=0894775e-b09c-4684-bf5d-84ddccc69567>

# sa_stack_score

## FPD10

In [91]:
# sa_stack_score

sq = """
with sa_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    alpha_stack_score ,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    alpha_stack_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from sa_stack_score;
"""

sa_stack_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

sa_stack_scorefpd10.head()

Job ID 7b23398b-286d-467b-b9e3-39bbe856debd successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,alpha_stack_score,ln_fpd10_flag,ln_mature_fpd10_flag
0,2024-07-16 19:09:27,a731f941-ed8a-4278-9ba4-1fd41132aa5e,0.069974,0,1
1,2024-05-02 17:24:11,9b118965-74ea-4832-a640-8b5ea2cc140e,0.019328,0,1
2,2024-09-15 09:21:50,2f8156e6-5c5b-454c-92d9-628291e6f99e,0.045933,0,1
3,2024-12-29 18:02:03,6548e088-8d8e-47ae-b33d-e6df00d80c38,0.054839,0,1
4,2025-01-27 12:33:16,107d411b-cd24-4dde-86a8-6f7eeeaaa7bb,0.016626,0,1


In [92]:
sa_stack_scorefpd10.head()

,disbursementdate,digitalLoanAccountId,alpha_stack_score,ln_fpd10_flag,ln_mature_fpd10_flag
0,2024-07-16 19:09:27,a731f941-ed8a-4278-9ba4-1fd41132aa5e,0.069974,0,1
1,2024-05-02 17:24:11,9b118965-74ea-4832-a640-8b5ea2cc140e,0.019328,0,1
2,2024-09-15 09:21:50,2f8156e6-5c5b-454c-92d9-628291e6f99e,0.045933,0,1
3,2024-12-29 18:02:03,6548e088-8d8e-47ae-b33d-e6df00d80c38,0.054839,0,1
4,2025-01-27 12:33:16,107d411b-cd24-4dde-86a8-6f7eeeaaa7bb,0.016626,0,1


In [93]:
gini_results = calculate_periodic_gini(sa_stack_scorefpd10, 'alpha_stack_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

The shape of dataframe after copy is:	(107, 7)


array(['start_date', 'end_date', 'alpha_stack_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [94]:
M1FPD10.describe()

,start_date,end_date,alpha_stack_score_FPD10_gini
count,107,107,107.000000
mean,2024-03-28 15:28:35.887850496,2024-04-08 01:07:17.383177472,0.373566
min,2023-05-29 00:00:00,2023-06-04 00:00:00,-0.292453
25%,2023-10-31 00:00:00,2023-11-08 12:00:00,0.333208
50%,2024-04-01 00:00:00,2024-04-07 00:00:00,0.374921
75%,2024-08-29 00:00:00,2024-09-04 12:00:00,0.453236
max,2025-01-27 00:00:00,2025-02-02 00:00:00,0.693277
std,NaN,NaN,0.137124


In [95]:
M1FPD10.head()

,start_date,end_date,alpha_stack_score_FPD10_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.579487,Week,alpha_stack_score,1.1.0,FPD10
1,2023-06-01,2023-06-30,-0.058077,Month,alpha_stack_score,1.1.0,FPD10
2,2023-06-05,2023-06-11,0.632035,Week,alpha_stack_score,1.1.0,FPD10
3,2023-06-12,2023-06-18,-0.292453,Week,alpha_stack_score,1.1.0,FPD10
4,2023-06-19,2023-06-25,0.103226,Week,alpha_stack_score,1.1.0,FPD10


## FPD30

In [96]:
# sa_stack_score

sq = """
with sa_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    alpha_stack_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    alpha_stack_score  is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from sa_stack_score;
"""

sa_stack_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

sa_stack_scorefpd30.head()

Job ID 45cecbc7-1c73-4225-bf11-42a9024681bf successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,alpha_stack_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2024-09-01 15:18:37,1a2cb6f2-e66f-444a-ba22-a05cb5652ef2,0.053850,0,1
1,2024-09-19 20:32:20,4f7bb0f5-ec5b-40c0-a145-6d8aa9b083ee,0.043131,1,1
2,2024-07-26 11:14:26,e42d366d-9370-4761-9d94-e7cee5834849,0.056931,0,1
3,2024-08-18 13:17:01,d1d7594f-fcac-4295-b037-d998766d91be,0.016098,0,1
4,2024-08-10 17:16:49,6846df09-7ffe-4106-91f6-3f9208dafbff,0.052733,0,1


In [97]:
sa_stack_scorefpd30.head()

,disbursementdate,digitalLoanAccountId,alpha_stack_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2024-09-01 15:18:37,1a2cb6f2-e66f-444a-ba22-a05cb5652ef2,0.053850,0,1
1,2024-09-19 20:32:20,4f7bb0f5-ec5b-40c0-a145-6d8aa9b083ee,0.043131,1,1
2,2024-07-26 11:14:26,e42d366d-9370-4761-9d94-e7cee5834849,0.056931,0,1
3,2024-08-18 13:17:01,d1d7594f-fcac-4295-b037-d998766d91be,0.016098,0,1
4,2024-08-10 17:16:49,6846df09-7ffe-4106-91f6-3f9208dafbff,0.052733,0,1


In [98]:
gini_results = calculate_periodic_gini(sa_stack_scorefpd30, 'alpha_stack_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

The shape of dataframe after copy is:	(104, 7)


array(['start_date', 'end_date', 'alpha_stack_score_FPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [99]:
M2FPD30.describe()

,start_date,end_date,alpha_stack_score_FPD30_gini
count,104,104,104.000000
mean,2024-03-20 01:36:55.384615424,2024-03-30 14:18:27.692307712,0.398293
min,2023-05-29 00:00:00,2023-06-04 00:00:00,-0.300469
25%,2023-10-28 06:00:00,2023-11-03 18:00:00,0.358042
50%,2024-03-21 12:00:00,2024-03-31 00:00:00,0.402276
75%,2024-08-13 18:00:00,2024-08-26 12:00:00,0.479677
max,2025-01-06 00:00:00,2025-01-31 00:00:00,0.784014
std,NaN,NaN,0.155443


In [100]:
M2FPD30.head()

,start_date,end_date,alpha_stack_score_FPD30_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.579487,Week,alpha_stack_score,1.1.0,FPD30
1,2023-06-01,2023-06-30,-0.059380,Month,alpha_stack_score,1.1.0,FPD30
2,2023-06-05,2023-06-11,0.632035,Week,alpha_stack_score,1.1.0,FPD30
3,2023-06-12,2023-06-18,-0.300469,Week,alpha_stack_score,1.1.0,FPD30
4,2023-06-19,2023-06-25,0.103226,Week,alpha_stack_score,1.1.0,FPD30


## FSTPD30

In [101]:
# sa_stack_score

sq = """
with sa_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    alpha_stack_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    alpha_stack_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from sa_stack_score;
"""

sa_stack_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

sa_stack_scorefspd30.head()

Job ID 72294047-2b35-4192-bc7f-4553aa0714e9 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,alpha_stack_score,ln_fspd30_flag,ln_mature_fspd30_flag
0,2024-07-16 19:09:27,a731f941-ed8a-4278-9ba4-1fd41132aa5e,0.069974,0,1
1,2024-05-02 17:24:11,9b118965-74ea-4832-a640-8b5ea2cc140e,0.019328,0,1
2,2024-09-15 09:21:50,2f8156e6-5c5b-454c-92d9-628291e6f99e,0.045933,0,1
3,2024-07-01 14:17:27,571c6877-62e0-4c47-a382-77b1aeb2840c,0.109843,0,1
4,2024-09-20 13:24:28,ea27f1dc-5bfa-4572-bf7f-994d9a98d848,0.036564,0,1


In [102]:
gini_results = calculate_periodic_gini(sa_stack_scorefspd30, 'alpha_stack_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
M3FSPD30.columns.values

The shape of dataframe after copy is:	(99, 7)


array(['start_date', 'end_date', 'alpha_stack_score_FSPD30_gini',
       'period', 'Model_Name', 'version', 'bad_rate'], dtype=object)

In [103]:
M3FSPD30.tail()

,start_date,end_date,alpha_stack_score_FSPD30_gini,period,Model_Name,version,bad_rate
94,2024-11-18,2024-11-24,0.366828,Week,alpha_stack_score,1.1.0,FSPD30
95,2024-11-25,2024-12-01,0.360872,Week,alpha_stack_score,1.1.0,FSPD30
96,2024-12-01,2024-12-31,0.393471,Month,alpha_stack_score,1.1.0,FSPD30
97,2024-12-02,2024-12-08,0.375955,Week,alpha_stack_score,1.1.0,FSPD30
98,2024-12-09,2024-12-15,0.430778,Week,alpha_stack_score,1.1.0,FSPD30


## FSTPD30

In [104]:
# sa_stack_score

sq = """
with sa_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    alpha_stack_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    alpha_stack_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from sa_stack_score;
"""

sa_stack_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

sa_stack_scorefstpd30.head()

Job ID c98a03b9-6b4f-40a2-b705-7771d5f5267b successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,alpha_stack_score,ln_fstpd30_flag,ln_mature_fstpd30_flag
0,2024-07-16 19:09:27,a731f941-ed8a-4278-9ba4-1fd41132aa5e,0.069974,0,1
1,2024-05-02 17:24:11,9b118965-74ea-4832-a640-8b5ea2cc140e,0.019328,0,1
2,2024-09-15 09:21:50,2f8156e6-5c5b-454c-92d9-628291e6f99e,0.045933,0,1
3,2024-07-01 14:17:27,571c6877-62e0-4c47-a382-77b1aeb2840c,0.109843,0,1
4,2024-09-20 13:24:28,ea27f1dc-5bfa-4572-bf7f-994d9a98d848,0.036564,0,1


In [105]:
sa_stack_scorefstpd30.tail()

,disbursementdate,digitalLoanAccountId,alpha_stack_score,ln_fstpd30_flag,ln_mature_fstpd30_flag
76861,2024-06-05 19:12:10,c43ff844-9406-4124-8a09-5ff284f5c7f7,0.132570,1,1
76862,2024-09-14 17:01:27,c405bfd6-437d-41c1-b6e3-8d1a1607811f,0.039156,1,1
76863,2024-09-21 13:11:19,2d1860fb-cb42-4c99-8121-294a9feb235b,0.036444,1,1
76864,2024-10-06 12:13:47,7e837c49-bfe0-4298-be76-5ac00d45187f,0.038666,1,1
76865,2024-11-06 18:16:04,4158dd38-80a4-47af-8e3a-3f8338ea68bc,0.109025,1,1


In [106]:
gini_results = calculate_periodic_gini(sa_stack_scorefstpd30, 'alpha_stack_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
M4FSTPD30.columns.values

The shape of dataframe after copy is:	(93, 7)


array(['start_date', 'end_date', 'alpha_stack_score_FSTPD30_gini',
       'period', 'Model_Name', 'version', 'bad_rate'], dtype=object)

## Combining the dataframes

In [107]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'alpha_stack_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate',
       'alpha_stack_score_FPD30_gini', 'alpha_stack_score_FSPD30_gini',
       'alpha_stack_score_FSTPD30_gini'], dtype=object)

In [108]:
final_df = final_df[['start_date', 'end_date', 'period', 'Model_Name', 'version', 'bad_rate', 'sa_stack_score_FPD10_gini', 'sa_stack_score_FPD30_gini',  'sa_stack_score_FSPD30_gini', 'sa_stack_score_FSTPD30_gini']].copy()

KeyError: "['sa_stack_score_FPD10_gini', 'sa_stack_score_FPD30_gini', 'sa_stack_score_FSPD30_gini', 'sa_stack_score_FSTPD30_gini'] not in index"

## Creating the table 

In [110]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_sa_stack_score;"""

client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=b3ffe802-82ca-4390-b413-492ea6fda602>

In [111]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('alpha_stack_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('alpha_stack_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('alpha_stack_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('alpha_stack_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_sa_stack_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=4d36f140-046a-46e5-ba6b-b9a33fbeef87>

# gen_credo_score

## FPD10

In [112]:


sq = """
with gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_gen_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
     risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    credo_gen_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from gen_credo_score;
"""

gen_credo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

gen_credo_scorefpd10.head()

Job ID af52157a-d5ad-4f3c-bec4-2241d01edcfa successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,credo_gen_score,ln_fpd10_flag,ln_mature_fpd10_flag
0,2024-09-01 15:18:37,1a2cb6f2-e66f-444a-ba22-a05cb5652ef2,0.113903,0,1
1,2024-12-15 12:58:01,950353da-8475-4d52-964b-e334dcbd95dc,0.190540,0,1
2,2024-07-27 19:43:54,cae2c508-fe59-47b9-8eef-fc5f6f08a184,0.068098,0,1
3,2023-02-02 17:24:09,538a175e-1dcc-4cd9-8527-c1b0b6bf2ef3,0.039439,0,1
4,2024-06-19 20:08:28,210afa02-21e2-4ecc-8e28-06969222c968,0.145484,0,1


In [113]:
gini_results = calculate_periodic_gini(gen_credo_scorefpd10, 'credo_gen_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

The shape of dataframe after copy is:	(134, 7)


array(['start_date', 'end_date', 'credo_gen_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [114]:
M1FPD10.head()

,start_date,end_date,credo_gen_score_FPD10_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.246725,Month,credo_gen_score,1.1.0,FPD10
1,2023-01-02,2023-01-08,0.301125,Week,credo_gen_score,1.1.0,FPD10
2,2023-01-09,2023-01-15,0.273400,Week,credo_gen_score,1.1.0,FPD10
3,2023-01-16,2023-01-22,0.667532,Week,credo_gen_score,1.1.0,FPD10
4,2023-01-23,2023-01-29,0.086505,Week,credo_gen_score,1.1.0,FPD10


## FPD30

In [115]:
sq = """
with gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_gen_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    credo_gen_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from gen_credo_score;
"""

gen_credo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

gen_credo_scorefpd30.head()

Job ID 154643f8-ab11-41e8-b7d4-4e36be1098a2 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,credo_gen_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2024-07-16 19:09:27,a731f941-ed8a-4278-9ba4-1fd41132aa5e,0.185556,0,1
1,2023-12-03 17:15:55,8effa90f-b747-40b9-b42f-39691224f596,0.183422,1,1
2,2024-02-26 15:08:47,acebf8de-bd42-4b00-83bc-d7c0999a7488,0.110977,0,1
3,2024-05-02 17:24:11,9b118965-74ea-4832-a640-8b5ea2cc140e,0.097561,0,1
4,2024-09-15 09:21:50,2f8156e6-5c5b-454c-92d9-628291e6f99e,0.134108,0,1


In [116]:
gen_credo_scorefpd30.head()

,disbursementdate,digitalLoanAccountId,credo_gen_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2024-07-16 19:09:27,a731f941-ed8a-4278-9ba4-1fd41132aa5e,0.185556,0,1
1,2023-12-03 17:15:55,8effa90f-b747-40b9-b42f-39691224f596,0.183422,1,1
2,2024-02-26 15:08:47,acebf8de-bd42-4b00-83bc-d7c0999a7488,0.110977,0,1
3,2024-05-02 17:24:11,9b118965-74ea-4832-a640-8b5ea2cc140e,0.097561,0,1
4,2024-09-15 09:21:50,2f8156e6-5c5b-454c-92d9-628291e6f99e,0.134108,0,1


In [117]:
gini_results = calculate_periodic_gini(gen_credo_scorefpd30, 'credo_gen_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

The shape of dataframe after copy is:	(131, 7)


array(['start_date', 'end_date', 'credo_gen_score_FPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [118]:
M2FPD30.head()

,start_date,end_date,credo_gen_score_FPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.253005,Month,credo_gen_score,1.1.0,FPD30
1,2023-01-02,2023-01-08,0.342149,Week,credo_gen_score,1.1.0,FPD30
2,2023-01-09,2023-01-15,0.407163,Week,credo_gen_score,1.1.0,FPD30
3,2023-01-16,2023-01-22,0.667532,Week,credo_gen_score,1.1.0,FPD30
4,2023-01-23,2023-01-29,0.025547,Week,credo_gen_score,1.1.0,FPD30


## FSPD30

In [119]:
sq = """
with gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_gen_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    credo_gen_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from gen_credo_score;
"""

gen_credo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(gen_credo_scorefspd30.head(2))

gini_results = calculate_periodic_gini(gen_credo_scorefspd30, 'credo_gen_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

Job ID f36d1b11-ea1f-466f-b688-6abe8a56f14c successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  credo_gen_score  \
0 2024-07-16 19:09:27  a731f941-ed8a-4278-9ba4-1fd41132aa5e         0.185556   
1 2024-02-26 15:08:47  acebf8de-bd42-4b00-83bc-d7c0999a7488         0.110977   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               0                      1  
1               0                      1  
The shape of dataframe after copy is:	(126, 7)
['start_date' 'end_date' 'credo_gen_score_FSPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,credo_gen_score_FSPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.172055,Month,credo_gen_score,1.1.0,FSPD30
1,2023-01-02,2023-01-08,0.262803,Week,credo_gen_score,1.1.0,FSPD30
2,2023-01-09,2023-01-15,0.189744,Week,credo_gen_score,1.1.0,FSPD30
3,2023-01-16,2023-01-22,0.410738,Week,credo_gen_score,1.1.0,FSPD30
4,2023-01-23,2023-01-29,0.073579,Week,credo_gen_score,1.1.0,FSPD30


## FSTPD30

In [120]:
sq = """
with gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_gen_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    credo_gen_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from gen_credo_score;
"""

gen_credo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(gen_credo_scorefstpd30.head(2))

gini_results = calculate_periodic_gini(gen_credo_scorefstpd30, 'credo_gen_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()

Job ID 80f47d6a-950f-4b19-9db8-8a9b20a29d0d successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  credo_gen_score  \
0 2024-09-01 15:18:37  1a2cb6f2-e66f-444a-ba22-a05cb5652ef2         0.113903   
1 2024-07-27 19:43:54  cae2c508-fe59-47b9-8eef-fc5f6f08a184         0.068098   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                0                       1  
1                0                       1  
The shape of dataframe after copy is:	(120, 7)
['start_date' 'end_date' 'credo_gen_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,credo_gen_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.180837,Month,credo_gen_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.282959,Week,credo_gen_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.189908,Week,credo_gen_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,0.198840,Week,credo_gen_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,0.162845,Week,credo_gen_score,1.1.0,FSTPD30


## Combining tables 

In [121]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'credo_gen_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'credo_gen_score_FPD30_gini',
       'credo_gen_score_FSPD30_gini', 'credo_gen_score_FSTPD30_gini'],
      dtype=object)

In [122]:
final_df = final_df[['start_date', 'end_date','period',
       'Model_Name', 'version', 'bad_rate', 'credo_gen_score_FPD10_gini','credo_gen_score_FPD30_gini',   'credo_gen_score_FSPD30_gini', 'credo_gen_score_FSTPD30_gini']].copy()

In [123]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_gen_credo_score;"""

client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=d8b25b69-ac54-4e51-86ce-06f5167e0194>

In [124]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('credo_gen_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('credo_gen_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('credo_gen_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('credo_gen_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_gen_credo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=025f741c-9cfb-41fa-bb2f-1f1166edf554>

# c_credo_score

In [125]:
sq = """
with c_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_cash_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    credo_cash_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from c_credo_score;
"""

c_credo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

c_credo_scorefpd10.head()

gini_results = calculate_periodic_gini(c_credo_scorefpd10, 'credo_cash_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with c_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_cash_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    credo_cash_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from c_credo_score;
"""

c_credo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

c_credo_scorefpd30.head()

gini_results = calculate_periodic_gini(c_credo_scorefpd30, 'credo_cash_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with c_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_cash_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
   risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    credo_cash_score  is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from c_credo_score;
"""

c_credo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(c_credo_scorefspd30.head(2))

gini_results = calculate_periodic_gini(c_credo_scorefspd30, 'credo_cash_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with c_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_cash_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    credo_cash_score  is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from c_credo_score;
"""

c_credo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(c_credo_scorefstpd30.head(2))

gini_results = calculate_periodic_gini(c_credo_scorefstpd30, 'credo_cash_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID 9891ff3c-ae90-438e-8409-a5ae78f32649 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(134, 7)
Job ID 807019f9-d018-45e3-aacd-27b2854dbb0b successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(131, 7)
Job ID e4295f6e-bbb6-4d03-92c8-a1d9ab73a0a0 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  credo_cash_score  \
0 2024-07-16 19:09:27  a731f941-ed8a-4278-9ba4-1fd41132aa5e          0.375585   
1 2024-02-26 15:08:47  acebf8de-bd42-4b00-83bc-d7c0999a7488          0.276160   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               0                      1  
1               0                      1  
The shape of dataframe after copy is:	(126, 7)
['start_date' 'end_date' 'credo_cash_score_FSPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']
Job ID 07ab0537-43a1-45fd-acf6-3f273988ea8b successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  credo_cash_score  \
0 2024-09-01 15:18:37  1a2cb6f2-e66f-444a-ba22-a05cb5652ef2          0.130117   
1 2024-07-27 19:43:54  cae2c508-fe59-47b9-8eef-fc5f6f08a184          0.154694   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                0                       1  
1                0                       1  
The shape of dataframe after copy is:	(120, 7)
['start_date' 'end_date' 'credo_cash_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,credo_cash_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.035948,Month,credo_cash_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.036009,Week,credo_cash_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.062143,Week,credo_cash_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,0.275891,Week,credo_cash_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,-0.050558,Week,credo_cash_score,1.1.0,FSTPD30


In [126]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'credo_cash_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'credo_cash_score_FPD30_gini',
       'credo_cash_score_FSPD30_gini', 'credo_cash_score_FSTPD30_gini'],
      dtype=object)

In [127]:
final_df = final_df[['start_date', 'end_date','period',    'Model_Name', 'version', 'bad_rate',
                     'credo_cash_score_FPD10_gini', 
                     'credo_cash_score_FPD30_gini', 
                     'credo_cash_score_FSPD30_gini',
                     'credo_cash_score_FSTPD30_gini']].copy()

In [128]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_c_credo_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=6a56a17e-5bcd-4f2c-9353-0aaf81f7e2cf>

In [129]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('credo_cash_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('credo_cash_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('credo_cash_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('credo_cash_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_c_credo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=2b042799-6031-42eb-a4a7-5c5482a56cdd>

# s_credo_score

In [130]:


sq = """
with s_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_sil_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    credo_sil_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from s_credo_score;
"""

s_credo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

s_credo_scorefpd10.head()

gini_results = calculate_periodic_gini(s_credo_scorefpd10, 'credo_sil_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with s_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_sil_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
   risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    credo_sil_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from s_credo_score;
"""

s_credo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

s_credo_scorefpd30.head()

gini_results = calculate_periodic_gini(s_credo_scorefpd30, 'credo_sil_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with s_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_sil_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    credo_sil_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from s_credo_score;
"""

s_credo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(s_credo_scorefspd30.head(2))

gini_results = calculate_periodic_gini(s_credo_scorefspd30, 'credo_sil_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with s_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_sil_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    credo_sil_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from s_credo_score;
"""

s_credo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(s_credo_scorefstpd30.head(2))

gini_results = calculate_periodic_gini(s_credo_scorefstpd30, 'credo_sil_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID 33b1d7ef-c083-4ab8-a111-6c3981d4ad23 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(134, 7)
Job ID dd115bc9-237c-4208-b0db-40f9fd866c0d successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(131, 7)
Job ID 1b05bc56-000a-4ef7-970b-0ceae1f7a511 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  credo_sil_score  \
0 2023-01-09 20:02:29  851fb2f2-3a0b-4abe-a89b-6f34e183ea08         0.112890   
1 2023-03-04 11:59:14  5d0bcd27-ceed-4309-8cf5-9fac9dfc2676         0.114866   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               0                      1  
1               0                      1  
The shape of dataframe after copy is:	(123, 7)
['start_date' 'end_date' 'credo_sil_score_FSPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']
Job ID 9af1e983-80c4-4b16-b83a-5d1a74b14664 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  credo_sil_score  \
0 2023-01-09 20:02:29  851fb2f2-3a0b-4abe-a89b-6f34e183ea08         0.112890   
1 2023-03-04 11:59:14  5d0bcd27-ceed-4309-8cf5-9fac9dfc2676         0.114866   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                0                       1  
1                0                       1  
The shape of dataframe after copy is:	(117, 7)
['start_date' 'end_date' 'credo_sil_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,credo_sil_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.219086,Month,credo_sil_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.261708,Week,credo_sil_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.251056,Week,credo_sil_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,0.275062,Week,credo_sil_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,0.157554,Week,credo_sil_score,1.1.0,FSTPD30


## Combining data

In [131]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'credo_sil_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'credo_sil_score_FPD30_gini',
       'credo_sil_score_FSPD30_gini', 'credo_sil_score_FSTPD30_gini'],
      dtype=object)

In [132]:
final_df = final_df[['start_date', 'end_date','period',    'Model_Name', 'version', 'bad_rate',
                     'credo_sil_score_FPD10_gini', 
                     'credo_sil_score_FPD30_gini',  
                     'credo_sil_score_FSPD30_gini',
                     'credo_sil_score_FSTPD30_gini']].copy()

In [133]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_s_credo_score;"""

client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=4ef451c1-67ec-4085-b71b-4513ff00fa44>

In [134]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('credo_sil_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('credo_sil_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('credo_sil_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('credo_sil_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_s_credo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=50ce80c8-e2a7-4498-9e4c-03b8a1e81e1a>

In [135]:
final_df.head()

,start_date,end_date,period,Model_Name,version,bad_rate,credo_sil_score_FPD10_gini,credo_sil_score_FPD30_gini,credo_sil_score_FSPD30_gini,credo_sil_score_FSTPD30_gini
0,2023-01-01,2023-01-31,Month,credo_sil_score,1.1.0,FPD10,0.230181,NaN,NaN,NaN
1,2023-01-02,2023-01-08,Week,credo_sil_score,1.1.0,FPD10,0.248978,NaN,NaN,NaN
2,2023-01-09,2023-01-15,Week,credo_sil_score,1.1.0,FPD10,0.296337,NaN,NaN,NaN
3,2023-01-16,2023-01-22,Week,credo_sil_score,1.1.0,FPD10,0.529870,NaN,NaN,NaN
4,2023-01-23,2023-01-29,Week,credo_sil_score,1.1.0,FPD10,0.079585,NaN,NaN,NaN


# fu_credo_score

In [136]:


sq = """
with fu_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_flexup_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    credo_flexup_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from fu_credo_score;
"""

fu_credo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

fu_credo_scorefpd10.head()

gini_results = calculate_periodic_gini(fu_credo_scorefpd10, 'credo_flexup_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with fu_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_flexup_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    credo_flexup_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from fu_credo_score;
"""

fu_credo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

fu_credo_scorefpd30.head()

gini_results = calculate_periodic_gini(fu_credo_scorefpd30, 'credo_flexup_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with fu_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_flexup_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    credo_flexup_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from fu_credo_score;
"""

fu_credo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(fu_credo_scorefspd30.head(2))

gini_results = calculate_periodic_gini(fu_credo_scorefspd30, 'credo_flexup_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with fu_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_flexup_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    credo_flexup_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from fu_credo_score;
"""

fu_credo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(fu_credo_scorefstpd30.head(2))

gini_results = calculate_periodic_gini(fu_credo_scorefstpd30, 'credo_flexup_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID bb7dfeee-9a77-46a9-9766-4f6e4f5c2461 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(134, 7)
Job ID 1df8cfbc-0d16-44de-8b5a-3eff7ce3c570 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(131, 7)
Job ID 894d8513-91a2-4e06-a05f-79557f254402 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  \
0 2024-09-01 15:18:37  1a2cb6f2-e66f-444a-ba22-a05cb5652ef2   
1 2024-07-27 19:43:54  cae2c508-fe59-47b9-8eef-fc5f6f08a184   

   credo_flexup_score  ln_fspd30_flag  ln_mature_fspd30_flag  
0            0.454018               0                      1  
1            0.154694               0                      1  
The shape of dataframe after copy is:	(126, 7)
['start_date' 'end_date' 'credo_flexup_score_FSPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']
Job ID 8922c774-2769-417b-89a9-57c6a4920d70 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  \
0 2024-09-01 15:18:37  1a2cb6f2-e66f-444a-ba22-a05cb5652ef2   
1 2024-07-27 19:43:54  cae2c508-fe59-47b9-8eef-fc5f6f08a184   

   credo_flexup_score  ln_fstpd30_flag  ln_mature_fstpd30_flag  
0            0.454018                0                       1  
1            0.154694                0                       1  
The shape of dataframe after copy is:	(120, 7)
['start_date' 'end_date' 'credo_flexup_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,credo_flexup_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.072860,Month,credo_flexup_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.002165,Week,credo_flexup_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.056674,Week,credo_flexup_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,0.326429,Week,credo_flexup_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,0.109935,Week,credo_flexup_score,1.1.0,FSTPD30


## Combining data

In [137]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'credo_flexup_score_FPD10_gini',
       'period', 'Model_Name', 'version', 'bad_rate',
       'credo_flexup_score_FPD30_gini', 'credo_flexup_score_FSPD30_gini',
       'credo_flexup_score_FSTPD30_gini'], dtype=object)

In [138]:
final_df = final_df[['start_date', 'end_date','period',    'Model_Name', 'version', 'bad_rate',
                     'credo_flexup_score_FPD10_gini', 'credo_flexup_score_FPD30_gini',    'credo_flexup_score_FSPD30_gini', 'credo_flexup_score_FSTPD30_gini']].copy()

In [139]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_fu_credo_score"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=20baf75f-9acb-46ac-afcd-9c6f3fb45c49>

In [140]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('credo_flexup_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('credo_flexup_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('v_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('credo_flexup_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_fu_credo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=4340fd76-129c-4441-a7c0-6813975a1c42>

In [141]:
final_df.head()

,start_date,end_date,period,Model_Name,version,bad_rate,credo_flexup_score_FPD10_gini,credo_flexup_score_FPD30_gini,credo_flexup_score_FSPD30_gini,credo_flexup_score_FSTPD30_gini
0,2023-01-01,2023-01-31,Month,credo_flexup_score,1.1.0,FPD10,0.023608,NaN,NaN,NaN
1,2023-01-02,2023-01-08,Week,credo_flexup_score,1.1.0,FPD10,-0.032464,NaN,NaN,NaN
2,2023-01-09,2023-01-15,Week,credo_flexup_score,1.1.0,FPD10,0.016278,NaN,NaN,NaN
3,2023-01-16,2023-01-22,Week,credo_flexup_score,1.1.0,FPD10,0.153247,NaN,NaN,NaN
4,2023-01-23,2023-01-29,Week,credo_flexup_score,1.1.0,FPD10,0.120242,NaN,NaN,NaN


# r_credo_score

In [142]:


sq = """
with r_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_reloan_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    credo_reloan_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from r_credo_score;
"""

r_credo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

r_credo_scorefpd10.head()

gini_results = calculate_periodic_gini(r_credo_scorefpd10, 'credo_reloan_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with r_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_reloan_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    credo_reloan_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from r_credo_score;
"""

r_credo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

r_credo_scorefpd30.head()

gini_results = calculate_periodic_gini(r_credo_scorefpd30, 'credo_reloan_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with r_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_reloan_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    credo_reloan_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from r_credo_score;
"""

r_credo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(r_credo_scorefspd30.head(2))

gini_results = calculate_periodic_gini(r_credo_scorefspd30, 'credo_reloan_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with r_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_reloan_score ,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    credo_reloan_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from r_credo_score;
"""

r_credo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(r_credo_scorefstpd30.head(2))

gini_results = calculate_periodic_gini(r_credo_scorefstpd30, 'credo_reloan_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID 7ad6ef53-f3a4-4abf-92b1-25d407161743 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(134, 7)
Job ID 83d03dde-84ac-4157-bd3d-ea76051818bc successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(131, 7)
Job ID b753d531-32fb-4412-83e9-5c801a5ecbe2 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  \
0 2024-09-01 15:18:37  1a2cb6f2-e66f-444a-ba22-a05cb5652ef2   
1 2024-07-27 19:43:54  cae2c508-fe59-47b9-8eef-fc5f6f08a184   

   credo_reloan_score  ln_fspd30_flag  ln_mature_fspd30_flag  
0            0.277636               0                      1  
1            0.068098               0                      1  
The shape of dataframe after copy is:	(126, 7)
['start_date' 'end_date' 'credo_reloan_score_FSPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']
Job ID 3a466496-f6ad-45ee-b4bc-b3d8f57b733c successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  \
0 2024-09-01 15:18:37  1a2cb6f2-e66f-444a-ba22-a05cb5652ef2   
1 2024-07-27 19:43:54  cae2c508-fe59-47b9-8eef-fc5f6f08a184   

   credo_reloan_score  ln_fstpd30_flag  ln_mature_fstpd30_flag  
0            0.277636                0                       1  
1            0.068098                0                       1  
The shape of dataframe after copy is:	(120, 7)
['start_date' 'end_date' 'credo_reloan_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,credo_reloan_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.048984,Month,credo_reloan_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.014955,Week,credo_reloan_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.193885,Week,credo_reloan_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,-0.019056,Week,credo_reloan_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,0.024691,Week,credo_reloan_score,1.1.0,FSTPD30


## Combining data

In [143]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'credo_reloan_score_FPD10_gini',
       'period', 'Model_Name', 'version', 'bad_rate',
       'credo_reloan_score_FPD30_gini', 'credo_reloan_score_FSPD30_gini',
       'credo_reloan_score_FSTPD30_gini'], dtype=object)

In [144]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate',
                     'credo_reloan_score_FPD10_gini',
                     'credo_reloan_score_FPD30_gini',
                     'credo_reloan_score_FSPD30_gini',
                     'credo_reloan_score_FSTPD30_gini']].copy()

In [145]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_r_credo_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=bc672678-a3c9-4b50-8159-b541799f3368>

In [146]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('credo_reloan_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('credo_reloan_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('credo_reloan_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('credo_reloan_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_r_credo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=891cf0e6-5d26-4c60-b57e-84be897ec623>

# old_gen_credo_score

In [147]:


sq = """
with old_gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_old_gen_score ,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    credo_old_gen_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from old_gen_credo_score;
"""

old_gen_credo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

old_gen_credo_scorefpd10.head()

gini_results = calculate_periodic_gini_threedigit(old_gen_credo_scorefpd10, 'credo_old_gen_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with old_gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_old_gen_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    credo_old_gen_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from old_gen_credo_score;
"""

old_gen_credo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

old_gen_credo_scorefpd30.head()

gini_results = calculate_periodic_gini_threedigit(old_gen_credo_scorefpd30, 'credo_old_gen_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with old_gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_old_gen_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    credo_old_gen_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from old_gen_credo_score;
"""

old_gen_credo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(old_gen_credo_scorefspd30.head(2))

gini_results = calculate_periodic_gini_threedigit(old_gen_credo_scorefspd30, 'credo_old_gen_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with old_gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_old_gen_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    credo_old_gen_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from old_gen_credo_score;
"""

old_gen_credo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(old_gen_credo_scorefstpd30.head(2))

gini_results = calculate_periodic_gini_threedigit(old_gen_credo_scorefstpd30, 'credo_old_gen_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID d6b8a49f-fe20-48f1-8fe4-5f8cfe50cc91 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(134, 7)
Job ID 5680b319-fef7-4893-a894-5383c2f7d087 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(131, 7)
Job ID 1a2a87c0-1750-44e1-bf4a-bb48e0004332 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  \
0 2023-12-03 17:15:55  8effa90f-b747-40b9-b42f-39691224f596   
1 2024-02-26 15:08:47  acebf8de-bd42-4b00-83bc-d7c0999a7488   

  credo_old_gen_score  ln_fspd30_flag  ln_mature_fspd30_flag  
0                 454               1                      1  
1                 468               0                      1  
The shape of dataframe after copy is:	(126, 7)
['start_date' 'end_date' 'credo_old_gen_score_FSPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']
Job ID 109d943f-8525-43ec-b04c-f4c7bfab66ba successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  \
0 2023-02-02 17:24:09  538a175e-1dcc-4cd9-8527-c1b0b6bf2ef3   
1 2024-06-30 14:13:02  b38b41bd-3cb8-4aa2-b233-7382e359e56a   

  credo_old_gen_score  ln_fstpd30_flag  ln_mature_fstpd30_flag  
0               496.0                0                       1  
1               396.0                0                       1  
The shape of dataframe after copy is:	(120, 7)
['start_date' 'end_date' 'credo_old_gen_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,credo_old_gen_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.120677,Month,credo_old_gen_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.230331,Week,credo_old_gen_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.145360,Week,credo_old_gen_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,0.137097,Week,credo_old_gen_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,0.000744,Week,credo_old_gen_score,1.1.0,FSTPD30


## Combining data

In [148]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'credo_old_gen_score_FPD10_gini',
       'period', 'Model_Name', 'version', 'bad_rate',
       'credo_old_gen_score_FPD30_gini',
       'credo_old_gen_score_FSPD30_gini',
       'credo_old_gen_score_FSTPD30_gini'], dtype=object)

In [149]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate',
                     'credo_old_gen_score_score_FPD10_gini',
                     'credo_old_gen_score_score_FPD30_gini',
                     'credo_old_gen_score_FSPD30_gini', 
                     'credo_old_gen_score_FSTPD30_gini']].copy()

KeyError: "['credo_old_gen_score_score_FPD10_gini', 'credo_old_gen_score_score_FPD30_gini'] not in index"

In [151]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_gen_credo_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=c89167cc-b35c-4cbb-a2d7-fbe86d9ac60e>

In [152]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('credo_old_gen_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('credo_old_gen_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('credo_old_gen_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('credo_old_gen_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_gen_credo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=82981c5a-b2c0-49ff-a41d-7003e64c4ed5>

In [153]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_gen_credo_score;"""

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID b07b5976-7e90-45eb-9110-efd7f747845c successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [154]:
df['bad_rate'].value_counts()

bad_rate
FPD10      134
FPD30      131
FSPD30     126
FSTPD30    120
Name: count, dtype: int64

# old_cic_score

In [155]:


sq = """
with old_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_cic_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    old_cic_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from old_cic_score;
"""

old_cic_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

old_cic_scorefpd10.head()

gini_results = calculate_periodic_hybrid_gini(old_cic_scorefpd10, 'old_cic_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with old_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_cic_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    old_cic_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from old_cic_score;
"""

old_cic_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

old_cic_scorefpd30.head()

gini_results = calculate_periodic_hybrid_gini(old_cic_scorefpd30, 'old_cic_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with old_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_cic_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    old_cic_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from old_cic_score;
"""

old_cic_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(old_cic_scorefspd30.head(2))

gini_results = calculate_periodic_hybrid_gini(old_cic_scorefspd30, 'old_cic_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with old_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_cic_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    old_cic_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from old_cic_score;
"""

old_cic_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(old_cic_scorefstpd30.head(2))

gini_results = calculate_periodic_hybrid_gini(old_cic_scorefstpd30, 'old_cic_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID 6e1245af-7819-420d-afb4-44e1eba5dfd5 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(94, 7)
Job ID 685a89b5-ccbe-4f26-ad4a-b0021bd787bf successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(91, 7)
Job ID 5e9e0d59-ab76-498d-9917-4ca707ca505a successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId old_cic_score  \
0 2024-08-02 10:57:00  4859467d-5a31-43ba-81a2-f0e39d55b360       521.000   
1 2024-09-03 14:53:40  72a5ffeb-defd-41df-bae6-f55df7be236b       605.000   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               1                      1  
1               1                      1  
The shape of dataframe after copy is:	(86, 7)
['start_date' 'end_date' 'old_cic_score_FSPD30_gini' 'period' 'Model_Name'
 'version' 'bad_rate']
Job ID 0382cbe7-964c-42e1-95f5-c42e257b583b successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId old_cic_score  \
0 2024-08-02 10:57:00  4859467d-5a31-43ba-81a2-f0e39d55b360       521.000   
1 2024-09-03 14:53:40  72a5ffeb-defd-41df-bae6-f55df7be236b       605.000   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                1                       1  
1                1                       1  
The shape of dataframe after copy is:	(80, 7)
['start_date' 'end_date' 'old_cic_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,old_cic_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-08-01,2023-08-31,0.014080,Month,old_cic_score,1.1.0,FSTPD30
1,2023-08-21,2023-08-27,-0.008276,Week,old_cic_score,1.1.0,FSTPD30
2,2023-08-28,2023-09-03,0.102646,Week,old_cic_score,1.1.0,FSTPD30
3,2023-09-01,2023-09-30,0.157024,Month,old_cic_score,1.1.0,FSTPD30
4,2023-09-04,2023-09-10,0.195623,Week,old_cic_score,1.1.0,FSTPD30


## Combining data

In [156]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'old_cic_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'old_cic_score_FPD30_gini',
       'old_cic_score_FSPD30_gini', 'old_cic_score_FSTPD30_gini'],
      dtype=object)

In [157]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate','old_cic_score_FPD10_gini','old_cic_score_FPD30_gini',    'old_cic_score_FSPD30_gini', 'old_cic_score_FSTPD30_gini']].copy()

In [158]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_cic_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=9015a3a7-1f9f-4d2e-a1c8-43f3d6f1bc5c>

In [159]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('old_cic_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('old_cic_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('old_cic_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('old_cic_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_cic_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=173bfb2e-92b3-42d0-871a-801f891e21ac>

# old_demo_score

In [160]:


sq = """
with old_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_demo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    old_demo_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from old_demo_score;
"""

old_demo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

old_demo_scorefpd10.head()

gini_results = calculate_periodic_hybrid_gini(old_demo_scorefpd10, 'old_demo_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with old_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_demo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    old_demo_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from old_demo_score;
"""

old_demo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

old_demo_scorefpd30.head()

gini_results = calculate_periodic_hybrid_gini(old_demo_scorefpd30, 'old_demo_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with old_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_demo_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    old_demo_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from old_demo_score;
"""

old_demo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(old_demo_scorefspd30.head(2))

gini_results = calculate_periodic_hybrid_gini(old_demo_scorefspd30, 'old_demo_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with old_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_demo_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    old_demo_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from old_demo_score;
"""

old_demo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(old_demo_scorefstpd30.head(2))

gini_results = calculate_periodic_hybrid_gini(old_demo_scorefstpd30, 'old_demo_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID c4b73ca0-4128-4b31-acf3-e28fa3f91a2d successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(134, 7)
Job ID d76181fc-68d5-468d-8177-1f7737c4d662 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(131, 7)
Job ID c6fd5698-4c50-47bd-82cb-9bb5468c0bcf successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  old_demo_score  \
0 2023-04-25 18:08:03  ee89ea72-76ab-4e1b-a0eb-3e959907db2b           405.0   
1 2024-09-15 10:57:25  350ad04c-6201-4208-88d6-013985c015ed           432.0   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               0                      1  
1               1                      1  
The shape of dataframe after copy is:	(126, 7)
['start_date' 'end_date' 'old_demo_score_FSPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']
Job ID 3b390552-dc1a-4b6f-89d3-4437ff0c2dea successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  old_demo_score  \
0 2024-09-01 15:18:37  1a2cb6f2-e66f-444a-ba22-a05cb5652ef2           442.0   
1 2023-02-02 17:24:09  538a175e-1dcc-4cd9-8527-c1b0b6bf2ef3           414.0   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                0                       1  
1                0                       1  
The shape of dataframe after copy is:	(120, 7)
['start_date' 'end_date' 'old_demo_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,old_demo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.125200,Month,old_demo_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.175128,Week,old_demo_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.046483,Week,old_demo_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,0.139188,Week,old_demo_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,0.131099,Week,old_demo_score,1.1.0,FSTPD30


## Combining data

In [161]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'old_demo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'old_demo_score_FPD30_gini',
       'old_demo_score_FSPD30_gini', 'old_demo_score_FSTPD30_gini'],
      dtype=object)

In [162]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate','old_demo_score_FPD10_gini','old_demo_score_FPD30_gini',    'old_demo_score_FSPD30_gini', 'old_demo_score_FSTPD30_gini']].copy()

In [163]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_demo_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e55b0704-b424-498c-b1fd-308238793b84>

In [164]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('old_demo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('old_demo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('old_demo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('old_demo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_demo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=a80fca21-1c62-4ebf-b17b-45e7e6039a14>

# bu_bureau_score

In [165]:
sq = """
with bu_bureau_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    bu_bureau_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    coalesce(bu_bureau_score, 0.0) > 0.0
  AND
    ln_mature_fpd10_flag = 1
)
select * from bu_bureau_score;
"""

bu_bureau_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

bu_bureau_scorefpd10.head()

gini_results = calculate_periodic_gini_threedigit(bu_bureau_scorefpd10, 'bu_bureau_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with bu_bureau_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    bu_bureau_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    coalesce(bu_bureau_score, 0.0) > 0.0
  AND
    ln_mature_fpd30_flag = 1
)
select * from bu_bureau_score;
"""

bu_bureau_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

bu_bureau_scorefpd30.head()

gini_results = calculate_periodic_gini_threedigit(bu_bureau_scorefpd30, 'bu_bureau_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with bu_bureau_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    bu_bureau_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    coalesce(bu_bureau_score, 0.0) > 0.0
  AND
    ln_mature_fspd30_flag = 1
)
select * from bu_bureau_score;
"""

bu_bureau_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(bu_bureau_scorefspd30.head(2))

gini_results = calculate_periodic_gini_threedigit(bu_bureau_scorefspd30, 'bu_bureau_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with bu_bureau_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    bu_bureau_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    coalesce(bu_bureau_score, 0.0) > 0.0
  AND
    ln_mature_fstpd30_flag = 1
)
select * from bu_bureau_score;
"""

bu_bureau_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(bu_bureau_scorefstpd30.head(2))

gini_results = calculate_periodic_gini_threedigit(bu_bureau_scorefstpd30, 'bu_bureau_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID c281ef11-f042-4e63-aa2c-e97ee3bb3f84 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(92, 7)
Job ID 4f97d764-e4c3-42b3-86cf-4df47e384a9d successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(92, 7)
Job ID 1287a3c0-942c-4229-8bfc-9e13c826b70c successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  bu_bureau_score  \
0 2024-05-10 18:47:17  b107da04-422e-47d9-9a28-178bc98ee3a2            287.0   
1 2024-04-03 19:55:18  a6e83c59-2ceb-4147-8fc3-e1f2cb5749a2            352.0   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               0                      1  
1               0                      1  
The shape of dataframe after copy is:	(92, 7)
['start_date' 'end_date' 'bu_bureau_score_FSPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']
Job ID 221c51d3-e505-4510-b30a-4e6338946ba4 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  bu_bureau_score  \
0 2024-05-10 18:47:17  b107da04-422e-47d9-9a28-178bc98ee3a2            287.0   
1 2024-04-03 19:55:18  a6e83c59-2ceb-4147-8fc3-e1f2cb5749a2            352.0   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                0                       1  
1                0                       1  
The shape of dataframe after copy is:	(92, 7)
['start_date' 'end_date' 'bu_bureau_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,bu_bureau_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.037500,Month,bu_bureau_score,1.1.0,FSTPD30
1,2023-01-09,2023-01-15,-0.066667,Week,bu_bureau_score,1.1.0,FSTPD30
2,2023-01-16,2023-01-22,-0.833333,Week,bu_bureau_score,1.1.0,FSTPD30
3,2023-01-23,2023-01-29,0.416667,Week,bu_bureau_score,1.1.0,FSTPD30
4,2023-01-30,2023-02-05,0.244444,Week,bu_bureau_score,1.1.0,FSTPD30


## Combining data

In [166]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'bu_bureau_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'bu_bureau_score_FPD30_gini',
       'bu_bureau_score_FSPD30_gini', 'bu_bureau_score_FSTPD30_gini'],
      dtype=object)

In [167]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate','bu_bureau_score_FPD10_gini','bu_bureau_score_FPD30_gini',    'bu_bureau_score_FSPD30_gini', 'bu_bureau_score_FSTPD30_gini']].copy()

In [168]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_bu_bureau_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=95d6f931-75ea-49af-b204-27c6bdd03374>

In [169]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('bu_bureau_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('bu_bureau_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('bu_bureau_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('bu_bureau_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_bu_bureau_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e03e250b-0c49-4246-a9b5-572729b7c568>